In [56]:

import ibmos2spark

# @hidden_cell
credentials = {
    'endpoint': 'https://s3-api.us-geo.objectstorage.service.networklayer.com',
    'api_key': 'QI0dGMLtvoha7CJ4aYeS3l6YOjWraYA8MMLcWBkQ0Jf2',
    'service_id': 'iam-ServiceId-a5dd711d-95a6-40de-972e-2d23e39d5d0c',
    'iam_service_endpoint': 'https://iam.ng.bluemix.net/oidc/token'}

configuration_name = 'os_d3956d7b54db4946b90a0cd557c288c3_configs'
cos = ibmos2spark.CloudObjectStorage(sc, credentials, configuration_name, 'bluemix_cos')

from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
df_data = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
    .option('inferSchema','True')\
  .load(cos.url('reduced_fico_accepted.csv', 'lendingclubpractice-donotdelete-pr-qmjuqkyixpbl38'))
df_data.take(5)


[Row(index=0, loan_amnt=15000, term=60, int_rate=12.39, installment=336.64, emp_length=10, dti=12.03, pub_rec_bankruptcies=0, log_annual_inc=4.892100171, fico_score=752.0, charged_off=0),
 Row(index=1, loan_amnt=10400, term=36, int_rate=6.99, installment=321.08, emp_length=8, dti=14.92, pub_rec_bankruptcies=0, log_annual_inc=4.763435481, fico_score=712.0, charged_off=1),
 Row(index=2, loan_amnt=21425, term=60, int_rate=15.59, installment=516.36, emp_length=6, dti=18.49, pub_rec_bankruptcies=0, log_annual_inc=4.804827486, fico_score=687.0, charged_off=0),
 Row(index=4, loan_amnt=7650, term=36, int_rate=13.66, installment=260.2, emp_length=0, dti=34.81, pub_rec_bankruptcies=0, log_annual_inc=4.69897869, fico_score=687.0, charged_off=1),
 Row(index=5, loan_amnt=9600, term=36, int_rate=13.66, installment=326.53, emp_length=10, dti=25.81, pub_rec_bankruptcies=0, log_annual_inc=4.838855385, fico_score=682.0, charged_off=0)]

Explore the loaded data by using the following Apache® Spark DataFrame methods:

    print schema
    print top ten records
    count all records



In [4]:
df_data.printSchema()

root
 |-- index: integer (nullable = true)
 |-- loan_amnt: integer (nullable = true)
 |-- term: integer (nullable = true)
 |-- int_rate: double (nullable = true)
 |-- installment: double (nullable = true)
 |-- emp_length: integer (nullable = true)
 |-- dti: double (nullable = true)
 |-- pub_rec_bankruptcies: integer (nullable = true)
 |-- log_annual_inc: double (nullable = true)
 |-- fico_score: double (nullable = true)
 |-- charged_off: integer (nullable = true)



In [5]:
df_data.dtypes

[('index', 'int'),
 ('loan_amnt', 'int'),
 ('term', 'int'),
 ('int_rate', 'double'),
 ('installment', 'double'),
 ('emp_length', 'int'),
 ('dti', 'double'),
 ('pub_rec_bankruptcies', 'int'),
 ('log_annual_inc', 'double'),
 ('fico_score', 'double'),
 ('charged_off', 'int')]

Handling missing values

In [6]:
df_data.where( df_data['emp_length'].isNull() ).count()

42253

In [7]:
#Filling the missing values in pub_rec_bankruptcies with 0
df_data = df_data.fillna(0, subset=['pub_rec_bankruptcies'])

In [8]:
df_data.where( df_data['pub_rec_bankruptcies'].isNull() ).count()

0

In [9]:
#Filling missing values with mean
from pyspark.sql.functions import avg

In [10]:
df_data_imputed = df_data 

In [11]:
for x in df_data_imputed.columns:
    meanValue = df_data.agg(avg(x)).first()[0]
    print (x,meanValue)
    df_data_imputed = df_data_imputed.fillna(meanValue, subset=['emp_length'])

index 652968.5496695648
loan_amnt 14315.458210325085
term 41.78736812656904
int_rate 13.49099271648095
installment 436.74962391997536
emp_length 5.973165375362512
dti 17.867718927541834
pub_rec_bankruptcies 0.13189429020866617
log_annual_inc 4.8108355738180135
fico_score 697.6032079078659
charged_off 0.20624157960995648


df_data.show()

In [12]:
df_data_imputed.where(df_data_imputed['emp_length'].isNull()).count()

0

In [13]:
#Check for null values in every column:
for x in df_data_imputed.columns:
    print (x, df_data_imputed.where(df_data_imputed[x].isNull()).count())

index 0
loan_amnt 0
term 0
int_rate 0
installment 0
emp_length 0
dti 36
pub_rec_bankruptcies 0
log_annual_inc 0
fico_score 0
charged_off 0


In [14]:
df_no_null = df_data_imputed.na.drop()

In [15]:
df_no_null = df_no_null.drop('index')

In [16]:
df_no_null

DataFrame[loan_amnt: int, term: int, int_rate: double, installment: double, emp_length: int, dti: double, pub_rec_bankruptcies: int, log_annual_inc: double, fico_score: double, charged_off: int]

In [17]:
#Check for null values in every column:
for x in df_no_null.columns:
    print (x, df_no_null.where(df_no_null[x].isNull()).count())

loan_amnt 0
term 0
int_rate 0
installment 0
emp_length 0
dti 0
pub_rec_bankruptcies 0
log_annual_inc 0
fico_score 0
charged_off 0


In [16]:
print (df_no_null.describe().show())

+-------+------------------+------------------+------------------+------------------+------------------+------------------+--------------------+------------------+------------------+------------------+
|summary|         loan_amnt|              term|          int_rate|       installment|        emp_length|               dti|pub_rec_bankruptcies|    log_annual_inc|        fico_score|       charged_off|
+-------+------------------+------------------+------------------+------------------+------------------+------------------+--------------------+------------------+------------------+------------------+
|  count|            814950|            814950|            814950|            814950|            814950|            814950|              814950|            814950|            814950|            814950|
|   mean|14315.475427940364| 41.78729983434567|13.490852469518304| 436.7497759617707| 33831.48133995951|17.867718927541834|  0.1318939812258421|4.8110477206376485| 697.6033633965274|0.20624332

In [22]:
df_no_null[df_no_null['charged_off']== 1].take(100)

[Row(loan_amnt=10400, term=36, int_rate=6.99, installment=321.08, emp_length=8, dti=14.92, pub_rec_bankruptcies=0, log_annual_inc=4.763435481, fico_score=712.0, charged_off=1),
 Row(loan_amnt=7650, term=36, int_rate=13.66, installment=260.2, emp_length=0, dti=34.81, pub_rec_bankruptcies=0, log_annual_inc=4.69897869, fico_score=687.0, charged_off=1),
 Row(loan_amnt=23325, term=36, int_rate=14.31, installment=800.71, emp_length=10, dti=27.03, pub_rec_bankruptcies=0, log_annual_inc=4.857338528, fico_score=667.0, charged_off=1),
 Row(loan_amnt=10000, term=36, int_rate=11.99, installment=332.1, emp_length=8, dti=8.44, pub_rec_bankruptcies=0, log_annual_inc=4.954247335, fico_score=677.0, charged_off=1)]

In [24]:
df_no_null.count()

814950

In [25]:
#Seeing how many distinct values per column
for col in df_no_null.columns:
    print(col, df_no_null.select(col).distinct().count())

loan_amnt 1482
term 2
int_rate 595
installment 71955
emp_length 12
dti 4945
pub_rec_bankruptcies 11
log_annual_inc 45928
fico_score 40
charged_off 2


SMOTE to balance the dataset
Not focusing here for now. WIll come back to it later


## Create Apache Spark Machine learning model

In [26]:
split_data = df_no_null.randomSplit([0.8, 0.20], 24)
train_data = split_data[0]
test_data = split_data[1]

print ("Number of training records: " + str(train_data.count()))
print ("Number of testing records : " + str(test_data.count()))

Number of training records: 652091
Number of testing records : 162859


In [27]:
#splitting data into test and val
split_data = train_data.randomSplit([0.8, 0.20], 24)
train_data = split_data[0]
val_data = split_data[1]

print ("Number of training records: " + str(train_data.count()))
print ("Number of validation records : " + str(val_data.count()))
print ("Total of training and val sets:"+ str(train_data.count() + val_data.count()))

Number of training records: 521801
Number of validation records : 130290
Total of training and val sets:652091


In [28]:
train_data.groupby('charged_off').show()

AttributeError: 'GroupedData' object has no attribute 'show'

In [29]:
train_data.describe().show()

+-------+------------------+------------------+------------------+-----------------+------------------+------------------+--------------------+-------------------+-----------------+-------------------+
|summary|         loan_amnt|              term|          int_rate|      installment|        emp_length|               dti|pub_rec_bankruptcies|     log_annual_inc|       fico_score|        charged_off|
+-------+------------------+------------------+------------------+-----------------+------------------+------------------+--------------------+-------------------+-----------------+-------------------+
|  count|            521801|            521801|            521801|           521801|            521801|            521801|              521801|             521801|           521801|             521801|
|   mean|14324.120258489347|  41.7842740814985|13.491563757063126|437.1161185394473|34114.350806150236|17.867728463532902|  0.1324087151998559|   4.81086238455416|697.5669019415448|0.206553839

In [30]:
# oversampling with reuse the data
dataInput_min = train_data[train_data['charged_off'] == 1]
dataInput_maj = train_data[train_data['charged_off'] == 0]
size_max = dataInput_maj.count()
size_min = dataInput_min.count()

while(size_min * 2 < size_max):
    dataInput_min = dataInput_min.union(dataInput_min)
    size_min = dataInput_min.count()
#     size_min = size_min *2
    print(size_min)
print(size_max - size_min)
tmp = dataInput_min.limit(size_max - size_min)
print(tmp.count())
dataInput_min = dataInput_min.union(tmp)

print(dataInput_min.count())
print(dataInput_maj.count())

tmp = dataInput_maj.union(dataInput_min)
train_data = tmp
tmp.describe().show()
# df.sample(False, 0.5, 42)
# dataInput_maj.select('charged_off').describe()

215560
198461
198461
414021
414021
+-------+------------------+------------------+------------------+------------------+------------------+------------------+--------------------+-------------------+------------------+-----------------+
|summary|         loan_amnt|              term|          int_rate|       installment|        emp_length|               dti|pub_rec_bankruptcies|     log_annual_inc|        fico_score|      charged_off|
+-------+------------------+------------------+------------------+------------------+------------------+------------------+--------------------+-------------------+------------------+-----------------+
|  count|            828042|            828042|            828042|            828042|            828042|            828042|              828042|             828042|            828042|           828042|
|   mean|14667.020573835627| 43.03862847536719|14.316787928636412|444.46013344734024| 38896.21358095363|18.653470488211937|  0.1374918180478768|  4.797582870

In [31]:
# dataInput_maj.describe().show()
# dataInput_min.describe().show()

In [32]:
# #SMOTE code
# oversample by SMOTE
# sm = SMOTE(random_state=100, ratio = 1.0)
# x_train_res, y_train_res = sm.fit_sample(training_features, training_target)
# print(len(x_train_res))
# print((y_train_res.mean())) #0.5
# x_t = x_train_res
# y_t = y_train_res

# import random
# import numpy as np
# from pyspark.sql import Row
# from sklearn import neighbors
# from pyspark.ml.feature import VectorAssembler

# def vectorizerFunction(dataInput, TargetFieldName):
#     if(dataInput.select(TargetFieldName).distinct().count() != 2):
#         raise ValueError("Target field must have only 2 distinct classes")
#     columnNames = list(dataInput.columns)
#     columnNames.remove(TargetFieldName)
#     dataInput = dataInput.select((','.join(columnNames)+','+TargetFieldName).split(','))
#     assembler=VectorAssembler(inputCols = columnNames, outputCol = 'features')
#     pos_vectorized = assembler.transform(dataInput)
#     vectorized = pos_vectorized.select('features',TargetFieldName).withColumn('label',pos_vectorized[TargetFieldName]).drop(TargetFieldName)
#     return vectorized

# def SmoteSampling(vectorized, k = 5, minorityClass = 1, majorityClass = 0, percentageOver = 200, percentageUnder = 100):
#     if(percentageUnder > 100|percentageUnder < 10):
#         raise ValueError("Percentage Under must be in range 10 - 100");
#     if(percentageOver < 100):
#         raise ValueError("Percentage Over must be in at least 100");
#     dataInput_min = vectorized[vectorized['label'] == minorityClass]
#     dataInput_maj = vectorized[vectorized['label'] == majorityClass]
#     feature = dataInput_min.select('features')
   
    
    
#     nbrs = neighbors.NearestNeighbors(n_neighbors=k, algorithm='auto').fit(feature)
#     neighbours =  nbrs.kneighbors(feature)
#     gap = neighbours[0]
#     neighbours = neighbours[1]
#     min_rdd = dataInput_min.drop('label').rdd
#     pos_rddArray = min_rdd.map(lambda x : list(x))
#     pos_ListArray = pos_rddArray.collect()
#     min_Array = list(pos_ListArray)
#     newRows = []
#     nt = len(min_Array)
#     nexs = percentageOver/100
#     for i in range(nt):
#         for j in range(nexs):
#             neigh = random.randint(1,k)
#             difs = min_Array[neigh][0] - min_Array[i][0]
#             newRec = (min_Array[i][0]+random.random()*difs)
#             newRows.insert(0,(newRec))
#     newData_rdd = sc.parallelize(newRows)
#     newData_rdd_new = newData_rdd.map(lambda x: Row(features = x, label = 1))
#     new_data = newData_rdd_new.toDF()
#     new_data_minor = dataInput_min.unionAll(new_data)
#     new_data_major = dataInput_maj.sample(False, (float(percentageUnder)/float(100)))
#     return new_data_major.unionAll(new_data_minor)


In [33]:
# train_data
tmp.groupBy('charged_off').count().show()

+-----------+------+
|charged_off| count|
+-----------+------+
|          1|414021|
|          0|414021|
+-----------+------+



In [34]:
train_data.dtypes

[('loan_amnt', 'int'),
 ('term', 'int'),
 ('int_rate', 'double'),
 ('installment', 'double'),
 ('emp_length', 'int'),
 ('dti', 'double'),
 ('pub_rec_bankruptcies', 'int'),
 ('log_annual_inc', 'double'),
 ('fico_score', 'double'),
 ('charged_off', 'int')]

In [35]:
#dataInput = spark.read.format('csv').options(header='true',inferSchema='true').load("sam.csv").dropna()
#SmoteSampling(vectorizerFunction(train_data, 'charged_off'), k = 2, minorityClass = 1, majorityClass = 0, percentageOver = 110, percentageUnder = 5)

In [36]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer, IndexToString, VectorAssembler, VectorIndexer
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml import Pipeline, Model

/usr/local/src/conda3_runtime/home/envs/DSX-Python35-Spark/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [37]:
# !pip install --user imblearn

In [38]:
from pyspark.sql.functions import log

In [39]:
stringIndexer_label = StringIndexer(inputCol="charged_off", outputCol="label").fit(df_no_null)
#stringIndexer_loan_amnt = StringIndexer(inputCol="loan_amnt", outputCol="loan_amnt_ix")
#stringIndexer_term = StringIndexer(inputCol="term", outputCol="term_ix")
#stringIndexer_int_rate = StringIndexer(inputCol="int_rate", outputCol="int_rate_ix")
#stringIndexer_installment = StringIndexer(inputCol="installment", outputCol="installment_ix")
#stringIndexer_emp_length = StringIndexer(inputCol="emp_length", outputCol="emp_length_ix")
#stringIndexer_annual_inc = StringIndexer(inputCol="log_annual_inc", outputCol="log_annual_inc_ix")
#stringIndexer_dti = StringIndexer(inputCol="dti", outputCol="dti_ix")
#stringIndexer_fico = StringIndexer(inputCol="fico_score", outputCol="fico_score_ix")
#stringIndexer_bankrupt = StringIndexer(inputCol="pub_rec_bankruptcies", outputCol="pub_rec_bankruptcies_ix")



In [40]:
vectorAssembler_features = VectorAssembler(inputCols=["loan_amnt", "term","int_rate", "installment","emp_length","log_annual_inc","dti","fico_score","pub_rec_bankruptcies"], outputCol="features")

In [41]:
labelConverter = IndexToString(inputCol="prediction", outputCol="predictedLabel", labels=stringIndexer_label.labels)


In [42]:
transform_df_pipeline = Pipeline(stages=[stringIndexer_label,
                              vectorAssembler_features])
transformed_df = transform_df_pipeline.fit(df_no_null).transform(df_no_null)
transformed_df.show()

+---------+----+--------+-----------+----------+-----+--------------------+--------------+----------+-----------+-----+--------------------+
|loan_amnt|term|int_rate|installment|emp_length|  dti|pub_rec_bankruptcies|log_annual_inc|fico_score|charged_off|label|            features|
+---------+----+--------+-----------+----------+-----+--------------------+--------------+----------+-----------+-----+--------------------+
|    15000|  60|   12.39|     336.64|        10|12.03|                   0|   4.892100171|     752.0|          0|  0.0|[15000.0,60.0,12....|
|    10400|  36|    6.99|     321.08|         8|14.92|                   0|   4.763435481|     712.0|          1|  1.0|[10400.0,36.0,6.9...|
|    21425|  60|   15.59|     516.36|         6|18.49|                   0|   4.804827486|     687.0|          0|  0.0|[21425.0,60.0,15....|
|     7650|  36|   13.66|      260.2|         0|34.81|                   0|    4.69897869|     687.0|          1|  1.0|[7650.0,36.0,13.6...|
|     9600|  

In [43]:
# test_data.describe().show()

In [44]:
from pyspark.mllib.tree import RandomForest
rf = RandomForestClassifier(numTrees = 100, maxDepth = 6, labelCol="label", featuresCol="features")
# rf = RandomForest.trainClassifier(trainingData, numClasses=2, categoricalFeaturesInfo={},
#                                      numTrees=3, featureSubsetStrategy="auto",
#                                      impurity='gini', maxDepth=4, maxBins=32)


In [45]:
pipeline_rf = Pipeline(stages=[stringIndexer_label, vectorAssembler_features, rf,labelConverter])

In [46]:
pipeline_rf


Pipeline_4b3bb8dab1082c11be9d

In [47]:
train_data.describe().show()


+-------+------------------+-----------------+------------------+-----------------+------------------+-----------------+--------------------+------------------+------------------+------------------+
|summary|         loan_amnt|             term|          int_rate|      installment|        emp_length|              dti|pub_rec_bankruptcies|    log_annual_inc|        fico_score|       charged_off|
+-------+------------------+-----------------+------------------+-----------------+------------------+-----------------+--------------------+------------------+------------------+------------------+
|  count|            828042|           828042|            828042|           828042|            828042|           828042|              828042|            828042|            828042|            828042|
|   mean|14667.020573835627|43.03862847536719|14.316787928635703|444.4601334473335| 38896.21358095363|18.65347048821191|  0.1374918180478768| 4.797582870806958| 694.4118172749692|               0.5|
| std

In [48]:
model_rf = pipeline_rf.fit(train_data)


In [51]:
def performance(predictions):
    predictionAndLabels = predictions.rdd.map(lambda lp: (lp.prediction, lp.label))
    from pyspark.mllib.evaluation import MulticlassMetrics
    metrics = MulticlassMetrics(predictionAndLabels)
    print("confusion matrix: ")
    print( metrics.confusionMatrix())

    labels = [0.0, 1.0]
    for label in sorted(labels):
        print("Class %s precision = %s" % (label, metrics.precision(label)))
        print("Class %s recall = %s" % (label, metrics.recall(label)))
        print("Class %s F1 Measure = %s" % (label, metrics.fMeasure(label, beta=1.0)))
        print()
print("test result") 

# predictions = model_rf.transform(test_data)
performance(predictions)

test result


/usr/local/src/conda3_runtime/home/envs/DSX-Python35-Spark/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


confusion matrix: 
DenseMatrix([[82122., 47219.],
             [11491., 22004.]])
Class 0.0 precision = 0.8772499546003226
Class 0.0 recall = 0.6349262801431874
Class 0.0 F1 Measure = 0.736672138647434

Class 1.0 precision = 0.31787122777111654
Class 1.0 recall = 0.6569338707269742
Class 1.0 F1 Measure = 0.4284351330828093



In [55]:
# (test_data.groupBy('charged_off')).count().show()
# +-----------+------+
# |charged_off| count|
# +-----------+------+
# |          1| 33484|
# |          0|129583|
# +-----------+------+
predictions[predictions['predictedLabel'] == 1].show()

+---------+----+--------+-----------+----------+-----+--------------------+--------------+----------+-----------+-----+--------------------+--------------------+--------------------+----------+--------------+
|loan_amnt|term|int_rate|installment|emp_length|  dti|pub_rec_bankruptcies|log_annual_inc|fico_score|charged_off|label|            features|       rawPrediction|         probability|prediction|predictedLabel|
+---------+----+--------+-----------+----------+-----+--------------------+--------------+----------+-----------+-----+--------------------+--------------------+--------------------+----------+--------------+
|     1000|  36|   12.49|      33.45|         3| 26.2|                   1|   4.612794449|     672.0|          0|  0.0|[1000.0,36.0,12.4...|[49.2609300547061...|[0.49260930054706...|       1.0|             1|
|     1000|  36|   12.49|      33.45|        10|23.13|                   0|   4.612794449|     677.0|          0|  0.0|[1000.0,36.0,12.4...|[49.8637361791496...|[0.

In [57]:
# compute train result 
train_vals = model_rf.transform(train_data)
print("validation result on train data ")      
performance(train_vals)

validation result on train data 
confusion matrix: 
DenseMatrix([[262951., 151039.],
             [143404., 270223.]])
Class 0.0 precision = 0.6461808021507331
Class 0.0 recall = 0.6351626850890119
Class 0.0 F1 Measure = 0.6406243718944941

Class 1.0 precision = 0.6414606586874677
Class 1.0 recall = 0.6533011626417038
Class 1.0 F1 Measure = 0.6473267703850452



In [58]:

# evaluatorRF = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
# accuracy = evaluatorRF.evaluate(predictions)
# metrics = MultilabelMetrics(scoreAndLabels)
# print("Accuracy = %g" % accuracy)

### Persist created model with watson-machine-learning-client

In [59]:
!pip install watson-machine-learning-client --upgrade

    100% |████████████████████████████████| 1.0MB 649kB/s eta 0:00:01
Requirement already up-to-date: tqdm in /gpfs/global_fs01/sym_shared/YPProdSpark/user/sdc3-31c6da65b79b47-718a49cedf78/.local/lib/python3.5/site-packages (from watson-machine-learning-client)
Requirement already up-to-date: tabulate in /usr/local/src/conda3_runtime.v39/home/envs/DSX-Python35-Spark/lib/python3.5/site-packages (from watson-machine-learning-client)
Requirement already up-to-date: urllib3 in /gpfs/global_fs01/sym_shared/YPProdSpark/user/sdc3-31c6da65b79b47-718a49cedf78/.local/lib/python3.5/site-packages (from watson-machine-learning-client)
Requirement already up-to-date: certifi in /gpfs/global_fs01/sym_shared/YPProdSpark/user/sdc3-31c6da65b79b47-718a49cedf78/.local/lib/python3.5/site-packages (from watson-machine-learning-client)
Requirement already up-to-date: pandas in /gpfs/global_fs01/sym_shared/YPProdSpark/user/sdc3-31c6da65b79b47-718a49cedf78/.local/lib/python3.5/site-packages (from watson-machin

In [60]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

/usr/local/src/conda3_runtime/home/envs/DSX-Python35-Spark/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/local/src/conda3_runtime/home/envs/DSX-Python35-Spark/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/local/src/conda3_runtime/home/envs/DSX-Python35-Spark/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
2018-08-03 16:19:03,150 - watson_machine_learning_client.metanames - WARNING - 'AUTHOR_EMAIL' meta prop is deprecated. It will be ignored.
2018-08-03 16:19:48,758 - watson_machine_learning_client.metanames - WARNING - 'AUTHOR_EMAIL' meta prop is deprecated. It will be ignored.
2018-08-03 16:21:04,826 - watson_machine_

In [61]:
wml_credentials={
  "username": "191a4d7e-10c4-456c-a6e7-8b35cf7fe71d",
  "password": "1de592a3-9d65-4458-8876-c5b310df37a7",
  "instance_id": "c549e573-c841-4eea-85be-764ef356f92c",
  "url": "https://ibm-watson-ml.mybluemix.net"
}

In [62]:
client = WatsonMachineLearningAPIClient(wml_credentials)


In [63]:
import json
instance_details = client.service_instance.get_details()

print(json.dumps(instance_details, indent=2))

{
  "metadata": {
    "created_at": "2018-06-05T15:44:20.699Z",
    "url": "https://ibm-watson-ml.mybluemix.net/v3/wml_instances/c549e573-c841-4eea-85be-764ef356f92c",
    "modified_at": "2018-08-03T20:29:55.916Z",
    "guid": "c549e573-c841-4eea-85be-764ef356f92c"
  },
  "entity": {
    "region": "us-south",
    "account": {
      "id": "500507492",
      "type": "PAYG",
      "name": ""
    },
    "deployments": {
      "url": "https://us-south.ml.cloud.ibm.com/v3/wml_instances/c549e573-c841-4eea-85be-764ef356f92c/deployments"
    },
    "source": "Bluemix",
    "status": "Active",
    "plan_id": "3f6acf43-ede8-413a-ac69-f8af3bb0cbfe",
    "published_models": {
      "url": "https://us-south.ml.cloud.ibm.com/v3/wml_instances/c549e573-c841-4eea-85be-764ef356f92c/published_models"
    },
    "organization_guid": "f71f2ab7-9571-469a-8af3-3d618275e667",
    "plan": "lite",
    "space_guid": "ed811b8a-f0ae-4c4b-a047-718a49cedf78",
    "usage": {
      "capacity_units": {
        "current"

In [65]:
model_props = {client.repository.ModelMetaNames.AUTHOR_NAME: "IBM", 
               client.repository.ModelMetaNames.AUTHOR_EMAIL: "ibm@ibm.com", 
               client.repository.ModelMetaNames.NAME: "Lending Assist Risk prediction Random forest model 9 features oversampling"}
published_model = client.repository.store_model(model=model_rf, pipeline=pipeline_rf, meta_props=model_props, training_data=train_data)



In [66]:
published_model_uid = client.repository.get_model_uid(published_model)
model_details = client.repository.get_details(published_model_uid)
print(json.dumps(model_details, indent=2))

{
  "metadata": {
    "created_at": "2018-08-03T21:19:53.061Z",
    "url": "https://us-south.ml.cloud.ibm.com/v3/wml_instances/c549e573-c841-4eea-85be-764ef356f92c/published_models/a1bac1d0-195c-4066-9c7a-049219beb905",
    "modified_at": "2018-08-03T21:19:53.115Z",
    "guid": "a1bac1d0-195c-4066-9c7a-049219beb905"
  },
  "entity": {
    "latest_version": {
      "created_at": "2018-08-03T21:19:53.115Z",
      "url": "https://us-south.ml.cloud.ibm.com/v3/ml_assets/models/a1bac1d0-195c-4066-9c7a-049219beb905/versions/cdd4954c-dbe5-4781-af1f-71bef5a6a6ab",
      "guid": "cdd4954c-dbe5-4781-af1f-71bef5a6a6ab"
    },
    "runtime_environment": "spark-2.1",
    "name": "Lending Assist Risk prediction Random forest model 9 features oversampling",
    "learning_configuration_url": "https://us-south.ml.cloud.ibm.com/v3/wml_instances/c549e573-c841-4eea-85be-764ef356f92c/published_models/a1bac1d0-195c-4066-9c7a-049219beb905/learning_configuration",
    "tags": [],
    "model_type": "mllib-2.1",

In [67]:
client.repository.list_models()

------------------------------------  --------------------------------------------------------------------------  ------------------------  ---------
GUID                                  NAME                                                                        CREATED                   FRAMEWORK
a1bac1d0-195c-4066-9c7a-049219beb905  Lending Assist Risk prediction Random forest model 9 features oversampling  2018-08-03T21:19:53.061Z  mllib-2.1
415afa2a-9aba-4519-b91e-9f54fba8f1ed  Lending Assist Risk prediction Random forest model 9 features               2018-08-03T21:19:16.399Z  mllib-2.1
cd475fd9-e511-4510-8bc8-c65771774ba6  ROI                                                                         2018-07-31T15:38:03.604Z  mllib-2.1
fdec002d-892d-4a24-8667-1ba7255149ba  Lending Assist Risk prediction Random forest model 9 features               2018-07-25T19:22:35.090Z  mllib-2.1
30d0ccf7-84be-4365-bbe3-7555908fb917  Heart Failure Prediction Model                                

In [68]:
loaded_model = client.repository.load(published_model_uid)

In [69]:
test_predictions = loaded_model.transform(test_data)

In [70]:
test_predictions.select('probability', 'predictedLabel').show(n=300, truncate=False)


+----------------------------------------+--------------+
|probability                             |predictedLabel|
+----------------------------------------+--------------+
|[0.72376667572176,0.27623332427824]     |0             |
|[0.6243062395801178,0.37569376041988234]|0             |
|[0.8312662036020618,0.16873379639793812]|0             |
|[0.851307088628548,0.14869291137145196] |0             |
|[0.6527058361414423,0.3472941638585577] |0             |
|[0.8419542964097299,0.1580457035902702] |0             |
|[0.6583392991368845,0.34166070086311556]|0             |
|[0.7284840843079727,0.2715159156920274] |0             |
|[0.7185777926587956,0.2814222073412044] |0             |
|[0.7667721040192279,0.23322789598077204]|0             |
|[0.8467725998541534,0.15322740014584654]|0             |
|[0.7433899927222927,0.25661000727770733]|0             |
|[0.6124830208268607,0.3875169791731393] |0             |
|[0.8241522152751066,0.17584778472489346]|0             |
|[0.8346453006

In [72]:
created_deployment = client.deployments.create(published_model_uid, name="Loan Status prediction oversampling deployed")




#######################################################################################

Synchronous deployment creation for uid: 'a1bac1d0-195c-4066-9c7a-049219beb905' started

#######################################################################################


INITIALIZING
DEPLOY_SUCCESS


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='5d1bbf2b-edaf-4bd6-86e0-da0f74ed46c4'
------------------------------------------------------------------------------------------------




In [73]:
scoring_endpoint = client.deployments.get_scoring_url(created_deployment)

print(scoring_endpoint)

https://ibm-watson-ml.mybluemix.net/v3/wml_instances/c549e573-c841-4eea-85be-764ef356f92c/deployments/5d1bbf2b-edaf-4bd6-86e0-da0f74ed46c4/online


In [74]:
client.deployments.list()

------------------------------------  --------------------------------------------  ------  --------------  ------------------------  ---------  -------------
GUID                                  NAME                                          TYPE    STATE           CREATED                   FRAMEWORK  ARTIFACT TYPE
5d1bbf2b-edaf-4bd6-86e0-da0f74ed46c4  Loan Status prediction oversampling deployed  online  DEPLOY_SUCCESS  2018-08-03T21:23:35.404Z  mllib-2.1  model
96328795-a18f-4ff6-af48-06bc2d992bce  test                                          online  DEPLOY_SUCCESS  2018-07-31T15:38:26.041Z  mllib-2.1  model
75725fc8-d1f6-4e16-b099-a872a4a8d673  Loan Status prediction                        online  DEPLOY_SUCCESS  2018-07-25T19:22:58.902Z  mllib-2.1  model
6ef42e2f-7a47-45e0-a2e3-6db26e433b50  Titianic_deploy                               online  DEPLOY_SUCCESS  2018-06-11T19:29:20.097Z  wml-1.1    model
b0730887-0921-4b27-a710-a2af2fa2d4e9  Titanic_deployment                      

In [75]:
payload_scoring = {"fields": ["loan_amnt", "term", "int_rate", "installment", "emp_length", "dti", "pub_rec_bankruptcies", "log_annual_inc", "fico_score"], "values": [}


SyntaxError: invalid syntax (<ipython-input-75-a5b9bcad1aeb>, line 1)